# Reading Data

In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import cv2
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models


class Normalize01(torch.nn.Module):
    """Normalizza i valori di un tensore tra 0 e 1."""
    def __init__(self):
        super().__init__()

    def forward(self, img):
        min_val, max_val = img.min(), img.max()
        return (img - min_val) / (max_val - min_val) if max_val > min_val else torch.zeros_like(img)

    def __repr__(self):
        return self.__class__.__name__ + '()'

def lambda_check_range(x):
    return (check_tensor_range(x), x)[1]

def check_tensor_range(img):
    """ Controlla se i valori del tensore sono nel range [0,1] dopo ToTensor. """
    if img.min() < 0 or img.max() > 1:
        print("Warning: Tensor values are out of range [0,1] after ToTensor")


class PollinateDataset(Dataset):
    def __init__(self, csv_file, dataset_folder, channels=('T', 'M', 'N'), selected_ids=None, transform=None):
        self.df = pd.read_csv(csv_file)
        self.dataset_folder = dataset_folder
        self.channels = channels
        self.transform = transform
        self.min_size = (100, 100)  # Dimensione minima richiesta
        # Use only selected IDs if provided
        if selected_ids is not None:  #lasciare a None per usare tutto il dataset
            self.df = self.df[self.df['folder'].isin(selected_ids)]

    def __len__(self):
        return len(self.df)

    def pad_image_to_min_size(self, image, min_size=(100, 100)):
        """Aggiunge padding all'immagine se è più piccola delle dimensioni minime richieste."""
        h, w = image.shape[:2]
        pad_h = max(0, min_size[0] - h)
        pad_w = max(0, min_size[1] - w)

        if pad_h > 0 or pad_w > 0:
            # Calcola il padding su ogni lato per centrare l'immagine
            top = pad_h // 2
            bottom = pad_h - top
            left = pad_w // 2
            right = pad_w - left

            image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)

        return image

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sample_id = str(row['folder'])
        label = row['class']
        folder_path = os.path.join(self.dataset_folder, sample_id)

        imgs = []
        for ch in self.channels:
            path = os.path.join(folder_path, f"{sample_id}_{ch}.npy")
            img = np.load(path)
            img = self.pad_image_to_min_size(img, self.min_size)
            imgs.append(img)

        img_stack = np.stack(imgs, axis=0)

        # Convert to float32 and normalize to [0,1] if not done already
        img_tensor = torch.from_numpy(img_stack).float()
        if self.transform:
            img_tensor = self.transform(img_tensor)
        else:
            #img_tensor = torch.from_numpy(img_stack).float()
            if img_tensor.max() > 1.0:
                img_tensor /= 255.0

        return row['folder'], img_tensor, label

class PollinateTestDataset(Dataset):
    def __init__(self, dataset_folder, channels=('T', 'M', 'N'), transform=None):
        self.dataset_folder = dataset_folder
        self.channels = channels
        self.transform = transform
        self.min_size = (100, 100)
        # Prendi solo le sottocartelle (ID)
        self.ids = sorted([name for name in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, name))])

    def __len__(self):
        return len(self.ids)

    def pad_image_to_min_size(self, image, min_size=(100, 100)):
        h, w = image.shape[:2]
        pad_h = max(0, min_size[0] - h)
        pad_w = max(0, min_size[1] - w)
        if pad_h > 0 or pad_w > 0:
            top = pad_h // 2
            bottom = pad_h - top
            left = pad_w // 2
            right = pad_w - left
            image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
        return image

    def __getitem__(self, idx):
        sample_id = self.ids[idx]
        folder_path = os.path.join(self.dataset_folder, sample_id)
        imgs = []
        for ch in self.channels:
            path = os.path.join(folder_path, f"{sample_id}_{ch}.npy")
            img = np.load(path)
            img = self.pad_image_to_min_size(img, self.min_size)
            imgs.append(img)
        img_stack = np.stack(imgs, axis=0)
        img_tensor = torch.from_numpy(img_stack).float()
        if self.transform:
            img_tensor = self.transform(img_tensor)
        else:
            if img_tensor.max() > 1.0:
                img_tensor /= 255.0
        return sample_id, img_tensor

dataset_folder = 'myFolder'
channels = ('T', 'M', 'N') #da modificare in base alla scelta dei canali

train_transform = transforms.Compose([
    transforms.Lambda(lambda_check_range),
    transforms.Resize((224, 224)),  # Porta tutto a 224x224 per il modello (ATTENZIONE: Da modificare in base alla scelta della soluzione)
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    Normalize01(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

testval_transform = transforms.Compose([
    transforms.Lambda(lambda_check_range),
    transforms.Resize((224, 224)),  # Porta tutto a 224x224 per il modello (ATTENZIONE: Da modificare in base alla scelta della soluzione)
    Normalize01(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



# Create train, test and validation

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


train_folder = os.path.join(dataset_folder, "train")

# 1. Estrai la lista delle acquisizioni (xxx)
all_folders = [name for name in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, name))]
all_acquisitions = sorted(list(set([folder.split('_')[0] for folder in all_folders])))

# 2. Suddividi le acquisizioni in 80% train, 10% val, 10% test
train_acq, temp_acq = train_test_split(all_acquisitions, test_size=0.2, random_state=42)
val_acq, test_acq = train_test_split(temp_acq, test_size=0.5, random_state=42)

print(f"Train acquisizioni: {len(train_acq)}, Val: {len(val_acq)}, Test: {len(test_acq)}")

# 3. Ottieni le celle (sottocartelle) per ogni split
train_ids = [folder for folder in all_folders if folder.split('_')[0] in train_acq]
val_ids = [folder for folder in all_folders if folder.split('_')[0] in val_acq]
test_ids = [folder for folder in all_folders if folder.split('_')[0] in test_acq]

print(f"Train celle: {len(train_ids)}, Val: {len(val_ids)}, Test: {len(test_ids)}")

# Ora puoi passare questi ID a PollinateDataset come selected_ids
train_dataset = PollinateDataset(
    csv_file=os.path.join(dataset_folder, 'train.csv'),
    dataset_folder=os.path.join(dataset_folder, "train"),
    channels=channels,
    selected_ids=train_ids,
    transform=train_transform
)
val_dataset = PollinateDataset(
    csv_file=os.path.join(dataset_folder, 'train.csv'),
    dataset_folder=os.path.join(dataset_folder, "train"),
    channels=channels,
    selected_ids=val_ids,
    transform=testval_transform
)
test_dataset = PollinateDataset(
    csv_file=os.path.join(dataset_folder, 'train.csv'),
    dataset_folder=os.path.join(dataset_folder, "train"),
    channels=channels,
    selected_ids=test_ids,
    transform=testval_transform
)


print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")



Train acquisizioni: 191, Val: 24, Test: 24
Train celle: 5585, Val: 619, Test: 594
Train dataset size: 5585
Validation dataset size: 619
Test dataset size: 594


# Network Definition

In [5]:
import torchvision.models as models
import torch.nn as nn

def get_pretrained_resnet(num_classes):
    model = models.resnet50(weights="IMAGENET1K_V1")
    # Adatta l'ultimo layer per il numero di classi
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Define the training function
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_acc = 0.0
    train_loss = 0.0
    for batch_idx, (ids, data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device).long()
        optimizer.zero_grad()
        output = model(data) #forward
        loss = criterion(output, target)  #calcolo loss
        loss.backward() #calcolo gradienti
        optimizer.step() #agg pesi

        train_loss += loss.item()
        train_acc += (output.argmax(dim=1) == target).sum().item()
    train_acc /= len(train_loader.dataset)
    train_loss /= len(train_loader.dataset)
    return train_loss, train_acc

# Define the validation function
def validate(model, device, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for ids, data, target in val_loader:
            data, target = data.to(device), target.to(device).long()
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            val_acc += (output.argmax(dim=1) == target).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)
    return val_loss, val_acc

# Train the Network

In [7]:
import matplotlib.pyplot as plt
input_dim = 64
output_dim = 10
lr = 5e-5
num_epochs = 50
batch_size = 16


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model, device, optimizer, and criterion
num_classes = len(set(train_dataset.df['class']))
model = get_pretrained_resnet(num_classes).to(device)

optimizer = optim.AdamW(model.parameters(), lr=lr)
# optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train_losses = []
train_accs = []
val_losses = []
val_accs = []
best_loss = 0.0

# Train the model
for epoch in range(num_epochs):
    # CREATE BATCHES
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    #1. WORK ON TRAIN SET
    # FOR EACH BATCH b:
    #     FORWARD STEP -> y_pred = MLP(b)
    #     COMPUTE THE LOSS -> Loss = criterion(y_pred, label)
    #     COMPUTE THE GRADIENTS
    #     UPDATE THE NETWORK
    #2. WORK ON VALIDATION SET
    # FOR EACH BATCH b:
    #     FORWARD STEP -> y_pred = MLP(b)
    #     COMPUTE THE LOSS -> Loss = criterion(y_pred, label)
    train_loss, train_acc = train(model, device, train_loader, optimizer, criterion, epoch)
    val_loss, val_acc = validate(model, device, val_loader, criterion)

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}')

    if epoch == 0 or val_loss<best_loss:
      best_loss = val_loss
      torch.save(model.state_dict(),  'best_weights.pth')

# Plot the training curves
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Training Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

KeyboardInterrupt: 

Final Training

In [ ]:
input_dim = 64
output_dim = 10
lr = 5e-5
num_epochs = 20
batch_size = 16

trainval_ids = train_ids + val_ids

trainval_dataset = PollinateDataset(
    csv_file=os.path.join(dataset_folder, 'train.csv'),
    dataset_folder=os.path.join(dataset_folder, 'train'),
    channels=channels,
    selected_ids=trainval_ids,
    transform=train_transform
)

# Initialize the model, device, optimizer, and criterion
num_classes = len(set(train_dataset.df['class']))
model = get_pretrained_resnet(num_classes).to(device)

optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# Load the training and validation data loaders

# Initialize the lists to store the training and validation losses and accuracies
train_losses = []
train_accs = []
val_losses = []
val_accs = []
best_loss = 0.0

# Train the model
for epoch in range(num_epochs):
    # CREATE BATCHES
    train_loader = DataLoader(trainval_dataset, batch_size=batch_size, shuffle=True)
    train_loss, train_acc = train(model, device, train_loader, optimizer, criterion, epoch)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')

torch.save(model.state_dict(),  'final_weights.pth')

Epoch 1/20, Train Loss: 0.0211, Train Acc: 0.8514
Epoch 2/20, Train Loss: 0.0132, Train Acc: 0.9172
Epoch 3/20, Train Loss: 0.0106, Train Acc: 0.9346
Epoch 4/20, Train Loss: 0.0096, Train Acc: 0.9429
Epoch 5/20, Train Loss: 0.0078, Train Acc: 0.9515
Epoch 6/20, Train Loss: 0.0070, Train Acc: 0.9597
Epoch 7/20, Train Loss: 0.0063, Train Acc: 0.9666
Epoch 8/20, Train Loss: 0.0055, Train Acc: 0.9650
Epoch 9/20, Train Loss: 0.0056, Train Acc: 0.9681
Epoch 10/20, Train Loss: 0.0046, Train Acc: 0.9731
Epoch 11/20, Train Loss: 0.0042, Train Acc: 0.9768
Epoch 12/20, Train Loss: 0.0044, Train Acc: 0.9728
Epoch 13/20, Train Loss: 0.0038, Train Acc: 0.9766
Epoch 14/20, Train Loss: 0.0033, Train Acc: 0.9810
Epoch 15/20, Train Loss: 0.0033, Train Acc: 0.9808
Epoch 16/20, Train Loss: 0.0025, Train Acc: 0.9865
Epoch 17/20, Train Loss: 0.0026, Train Acc: 0.9860
Epoch 18/20, Train Loss: 0.0029, Train Acc: 0.9828
Epoch 19/20, Train Loss: 0.0030, Train Acc: 0.9832
Epoch 20/20, Train Loss: 0.0025, Train A

# Compute the prediction on Test Set

In [ ]:
import pandas as pd

# Crea il dataset e loader
test_dataset = PollinateDataset(
    csv_file=os.path.join(dataset_folder, 'train.csv'),  
    dataset_folder=os.path.join(dataset_folder, 'train'),
    channels=channels,
    selected_ids=test_ids,
    transform=testval_transform
)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.load_state_dict(torch.load('final_weights.pth'))
model.eval()

test_predictions = []
true_labels = []
test_prob = []

with torch.no_grad():
    for ids, inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        _, predicted = torch.max(probs, 1)

        test_predictions.extend(predicted.cpu().numpy())
        test_prob.extend(probs.cpu().numpy())
        true_labels.extend(labels.numpy())

# Accuracy
acc = (np.array(test_predictions) == np.array(true_labels)).mean()
print("Test Accuracy:", acc)


Test Accuracy: 0.9612794612794613


In [9]:
print(outputs)
print(torch.nn.functional.softmax(outputs, dim=1))

tensor([[ -2.5505,   6.2949, -23.6103, -23.9749, -22.3061, -21.3416, -22.8903,
         -22.1036, -22.4021, -19.6203],
        [ -1.6605,   6.3218, -23.2537, -24.7811, -21.7051, -23.6336, -23.4781,
         -25.4486, -25.9040, -23.0606],
        [  3.7169,   0.0809, -29.0194, -29.0389, -27.8692, -27.4983, -29.2978,
         -30.1032, -31.4293, -29.7936],
        [  2.7896,   1.6813, -31.0087, -32.2892, -30.2858, -27.8040, -30.8533,
         -29.9769, -31.4373, -28.7290],
        [  1.3196,   4.4041, -29.8822, -28.6497, -27.6455, -26.2862, -28.9228,
         -29.7355, -30.5645, -26.9077],
        [ -2.3715,   7.9302, -46.5500, -46.9655, -45.7877, -47.1357, -45.3250,
         -45.0015, -48.7084, -45.7110],
        [ -3.3131,   6.2960, -17.6418, -17.2197, -16.3994, -15.2296, -16.8975,
         -16.4865, -16.5364, -14.0785],
        [  0.1601,   9.3648, -50.7262, -48.7365, -49.1076, -50.1266, -49.5135,
         -47.1522, -50.2207, -51.3048]])
tensor([[1.4403e-04, 9.9986e-01, 1.0287e-13, 7.

# Submission

In [ ]:
# Crea il dataset completo di training (usa tutto train.csv)
full_train_dataset = PollinateDataset(
    csv_file=os.path.join(dataset_folder, 'train.csv'),
    dataset_folder=os.path.join(dataset_folder, 'train'),
    channels=channels,
    transform=train_transform
)

full_train_loader = DataLoader(full_train_dataset, batch_size=16, shuffle=True)

# Initialize the model, device, optimizer, and criterion
num_classes = len(set(train_dataset.df['class']))
model = get_pretrained_resnet(num_classes).to(device)

# Si addestra l'intera rete

optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

## PROVARE A MODIFICARE ANCHE NELL'ALTRO
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
criterion = nn.CrossEntropyLoss()

num_epochs = 20
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, device, full_train_loader, optimizer, criterion, epoch)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')

torch.save(model.state_dict(), 'final_weights_full.pth')



Epoch 1/20, Train Loss: 0.0206, Train Acc: 0.8563
Epoch 2/20, Train Loss: 0.0134, Train Acc: 0.9172
Epoch 3/20, Train Loss: 0.0098, Train Acc: 0.9410
Epoch 4/20, Train Loss: 0.0084, Train Acc: 0.9520
Epoch 5/20, Train Loss: 0.0074, Train Acc: 0.9562
Epoch 6/20, Train Loss: 0.0068, Train Acc: 0.9594
Epoch 7/20, Train Loss: 0.0058, Train Acc: 0.9669
Epoch 8/20, Train Loss: 0.0052, Train Acc: 0.9698
Epoch 9/20, Train Loss: 0.0045, Train Acc: 0.9734
Epoch 10/20, Train Loss: 0.0041, Train Acc: 0.9751
Epoch 11/20, Train Loss: 0.0037, Train Acc: 0.9785
Epoch 12/20, Train Loss: 0.0038, Train Acc: 0.9772
Epoch 13/20, Train Loss: 0.0041, Train Acc: 0.9768
Epoch 14/20, Train Loss: 0.0034, Train Acc: 0.9801
Epoch 15/20, Train Loss: 0.0035, Train Acc: 0.9815
Epoch 16/20, Train Loss: 0.0029, Train Acc: 0.9835
Epoch 17/20, Train Loss: 0.0027, Train Acc: 0.9834
Epoch 18/20, Train Loss: 0.0026, Train Acc: 0.9862
Epoch 19/20, Train Loss: 0.0030, Train Acc: 0.9828
Epoch 20/20, Train Loss: 0.0029, Train A

In [11]:
import os
import pandas as pd

test_folder = os.path.join(dataset_folder, 'test')
test_dataset = PollinateTestDataset(
    dataset_folder=test_folder,
    channels=channels,
    transform=testval_transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.load_state_dict(torch.load('final_weights_full.pth'))
model.eval()

test_ids_out = []
test_predictions = []
test_prob = []

with torch.no_grad():
    for ids, inputs in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        _, predicted = torch.max(probs, 1)
        test_ids_out.extend(ids)
        test_predictions.extend(predicted.cpu().numpy())
        test_prob.extend(probs.cpu().numpy())

df_pred = pd.DataFrame({'Img': test_ids_out, 'class': test_predictions})
df_pred.to_csv('test_predictions4.csv', index=False)